In [3]:
import config
import pandas as pd
import utils


tasks = ["prostatex", "prostate-ucla"]

for task in tasks:
    result_dir = config.RESULT_DIR / task / "prostate"
    df = pd.read_csv(result_dir / "prostate_features.csv")
    print(len(df))
    df_selected = utils.add_label_col(df)
    print(df_selected['label'].value_counts())

199
True     69
False    29
Name: label, dtype: int64
950
True     502
False    243
Name: label, dtype: int64


/home/pw-research/git/radiomics-benchmark/prostate/utils.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = df["gleason_group"] > 1
/home/pw-research/git/radiomics-benchmark/prostate/utils.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = df["max_gleason"] > 6
